## Satellite imagery and Heritage Place geometry
> Notebook created by Will Deadman and Thomas Huet

<img src = "https://raw.githubusercontent.com/eamena-project/eamena-arches-dev/main/projects/caravanserail/image.png" width = "500">


Plot EAMENA Heritage Places polygon geometries using Earth Engine (`ee`) and use theses geometries to clip aerial photographs (ex: LANDSAT), label the aerial photograph with Heritage Place functions (ex: Caravanserais), disturbances, etc. to create a learning base

Load libraries

In [10]:
import pandas as pd
import ee
import requests
import json
import folium
from IPython.core.display import display, HTML

Google credentials to access GEE

In [2]:
ee.Authenticate()
ee.Initialize(project = "ee-zoometh")

Load the geometries of caravanserais

In [3]:
url = "https://raw.githubusercontent.com/eamena-project/eamena-arches-dev/main/projects/caravanserail/caravanserail.geojson"
response = requests.get(url)
geojson = json.loads(response.text)

Show a sample of the dataset in a tabular form

In [12]:
n=10
features = geojson['features']
data = []
for feature in features[:n]:
    properties = feature['properties']
    geometries = feature['geometry']
    entry = {
        'EAMENA ID': properties.get('EAMENA ID', None),
        'Site Feature Interpretation Type': properties.get('Site Feature Interpretation Type', None),
        'Overall Condition State Type': properties.get('Overall Condition State Type', None),
        'Geometry': geometries.get('coordinates', None)

    }
    data.append(entry)
pd.DataFrame(data)

,EAMENA ID,Site Feature Interpretation Type,Overall Condition State Type,Geometry
0,EAMENA-0182033,Caravanserai/Khan,Poor,"[[[59.974807, 29.88018], [59.975204, 29.880725..."
1,EAMENA-0182057,Caravanserai/Khan,Very Bad,"[55.05059, 36.42466]"
2,EAMENA-0164904,None,None,"[[[52.162675, 34.746338], [52.163281, 34.74621..."
3,EAMENA-0164906,None,None,"[[[52.179277, 34.759593], [52.179776, 34.75943..."
4,EAMENA-0164905,None,None,"[[[52.176537, 34.76554], [52.177347, 34.765694..."
5,EAMENA-0207223,Caravanserai/Khan,Fair,"[54.46618, 31.59461]"
6,EAMENA-0182056,Caravanserai/Khan,Fair,"[51.08376, 35.47244]"
7,EAMENA-0182056,Caravanserai/Khan,Fair,"[[[[51.08336, 35.4725], [51.08374, 35.47208], ..."
8,EAMENA-0182058,Caravanserai/Khan,Fair,"[52.73111, 35.25524]"
9,EAMENA-0182058,Caravanserai/Khan,Fair,"[[[[52.7309, 35.25559], [52.73066, 35.25505], ..."


Geometries used to clip

In [4]:
my_geom = 3 # select a geometry (1, 3, etc.) /!\ doesn't work with POINTS
geom_type = geojson['features'][my_geom]['geometry']['type']

if geom_type == 'Polygon':
  EAMENA_ID = geojson['features'][my_geom]['properties']['EAMENA ID']

  # coordinates
  hp_coords = geojson['features'][my_geom]['geometry']['coordinates']
  hp = ee.Geometry.Polygon(hp_coords)

  # centroid
  centroid = hp.centroid()
  centroid_info = centroid.getInfo()
  coord_x = centroid_info['coordinates'][0]
  coord_y = centroid_info['coordinates'][1]

  # aoi = bbox/mbr
  flat_coords = [item for sublist in hp_coords for item in sublist]
  min_lon, min_lat = min(flat_coords, key=lambda x: x[0])[0], min(flat_coords, key=lambda x: x[1])[1]
  max_lon, max_lat = max(flat_coords, key=lambda x: x[0])[0], max(flat_coords, key=lambda x: x[1])[1]
  AOI = ee.Geometry.BBox(min_lon, min_lat, max_lon, max_lat) # bbox

  # date
  Y1 = 2021
  Y2 = 2021
  M1 = 7
  M2 = M1 + 3

  # Sentinel-2 composite
  collection = (
      ee.ImageCollection("COPERNICUS/S2_SR")
      .filterBounds(AOI)
      .filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", 10)
      .filter(ee.Filter.calendarRange(Y1, Y2, "year"))
      .filter(ee.Filter.calendarRange(M1, M2, "month"))
      .select(["B.*"])
  )

  image = ee.Image(collection.median()).clip(AOI).divide(10000)
  print("Done")
else:
  print("The current geometry is " + geom_type)

Done


/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:204: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


Show Map

In [6]:
# Visualization parameters.
vis_params = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

# Folium map centered
map = folium.Map(location=[coord_y, coord_x], zoom_start=18, tiles=None)  # Set tiles=None to start with a blank map

# Google Satellite
google_satellite = folium.TileLayer(
    tiles='https://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Satellite',
    max_zoom=20,
    subdomains=['mt1', 'mt2', 'mt3'],
    overlay=False,
    control=True,
).add_to(map)

# HP shape
hp_geojson = ee.FeatureCollection([ee.Feature(hp)]).getInfo()
folium.GeoJson(
    data=hp_geojson,
    style_function=lambda x: {
        'color': 'blue',
        'weight': 1,
        'fillOpacity': 0,
    },
    name='HP Geometry'
).add_to(map)

# clipped geometry
map_id_dict = ee.Image(image).getMapId(vis_params)
folium.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    control=True,
    name='median composite',
).add_to(map)

# control panel
map.add_child(folium.LayerControl())

# Display the map.
map_html = map._repr_html_()
map_display = HTML(data=f'<div style="width: 800px; height: 800px;">{map_html}</div>')
display(map_display)


Export

In [ ]:
# export
task = ee.batch.Export.image.toDrive(
    image=image,
    description = EAMENA_ID,
    folder="gee_python",
    region=AOI,
    scale=100,
    crs="EPSG:4326",
)

task.start()

In [ ]:
task.status()

{'state': 'COMPLETED',
 'description': 'EAMENA-0164906',
 'creation_timestamp_ms': 1710155457515,
 'update_timestamp_ms': 1710155483587,
 'start_timestamp_ms': 1710155463673,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/#folders/1hxwSqjtmcAk2baSX0nvbW7ZT6OmuUEIn'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 1.1529306173324585,
 'id': 'N2ZUYMN3EEHOPXYAQFBDMAET',
 'name': 'projects/ee-zoometh/operations/N2ZUYMN3EEHOPXYAQFBDMAET'}